In [ ]:
import pydsm
from pydsm import postpro

# Dask related functions

In [ ]:
use_dask=False

import dask
from dask.distributed import Client, LocalCluster

class DaskCluster:
    def __init__(self):
        self.client=None
    def start_local_cluster(self):
        cluster = LocalCluster(n_workers=8, threads_per_worker=1, memory_limit='6G') # threads_per_worker=1 needed if using numba :(
        self.client = Client(cluster)
    def stop_local_cluster(self):
        self.client.shutdown()
        self.client=None

In [ ]:
if use_dask:
    cluster = DaskCluster()
    cluster.start_local_cluster()
    cluster.client

In [ ]:
def run_all(processors):
    tasks=[dask.delayed(postpro.run_processor)(processor,dask_key_name=f'{processor.study.name}::{processor.location.name}/{processor.vartype.name}') for processor in processors]
    dask.compute(tasks)

In [ ]:
base_dir='e:/dsm2v82_calibration/'
model_output_dir='output/'
study_file_map = {'DCD': base_dir+'BaselineRun/DSM2_DCD/'+model_output_dir+'historical_v81_DCD.dss',
                  'DCD1.2 SMCD': base_dir+'BaselineRun/DSM2_DCD1.2_SMCD/'+model_output_dir+'historical_v82b1.dss',
                  'DCD1.2 SMCD_last3': base_dir+'CalibrationRuns/last/DSM2_DCD1.2_SMCD_last3bin/'+model_output_dir+'historical_v82b1.dss',
                  'DICU': base_dir+'BaselineRun/DSM2_DICU/'+model_output_dir+'historical_v81_DICU.dss'}

# Setup for EC

In [ ]:
for study_name in study_file_map:
    dssfile=study_file_map[study_name]
    locationfile='e:/dsm2v82_calibration/LocationInfo/calibration_ec_stations.csv'
    vartype='EC'
    units='mmhos/cm'
    observed=False
    processors=postpro.build_processors(dssfile, locationfile, vartype, units, study_name, observed)
    print(f'Processing {vartype} for study: {study_name}')
    run_all(processors)
    if use_dask:
        run_all(processors)
    else:
        for p in processors:
           postpro.run_processor(p)

# Setup for FLOW

In [1]:
for study_name in study_file_map:    
    dssfile=study_file_map[study_name]
    locationfile='e:/dsm2v82_calibration/LocationInfo/calibration_flow_stations.csv'
    vartype='FLOW'
    units='cfs'
    observed=False
    processors=postpro.build_processors(dssfile, locationfile, vartype, units, study_name, observed)
    run_all(processors)
    if use_dask:
        run_all(processors)
    else:
        for p in processors:
           postpro.run_processor(p)

NameError: name 'study_file_map' is not defined

# Setup for STAGE

In [ ]:
for study_name in study_file_map:    
    dssfile=study_file_map[study_name]
    locationfile='e:/dsm2v82_calibration/LocationInfo/calibration_stage_stations.csv'
    vartype='STAGE'
    units='feet'
    observed=False
    processors=postpro.build_processors(dssfile, locationfile, vartype, units, study_name, observed)
    run_all(processors)
    if use_dask:
        run_all(processors)
    else:
        for p in processors:
           postpro.run_processor(p)

In [ ]:
if use_dask:
    cluster.stop_local_cluster()